In [1]:
'''
必要に応じて !pip install 
'''
import tensorflow as tf
import string
import re
import csv
import numpy as np
import random
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import files

In [2]:
'''
ここの部分は後々改良
'''
uploaded = files.upload()

Saving ja-en.csv to ja-en (1).csv


In [4]:
#!pwd
!ls

ja-en.csv  sample_data


In [3]:
#データ数増やす場合はgoogle driveに
#csvかtxt..etc..


name_csv = "ja-en.csv"

ja_train_text = []
en_train_text = []
train_pairs = []
val_pairs = []

with open(name_csv, newline='') as csvfile:
  reader = csv.DictReader(csvfile)
  
  for row in reader:
    ja_train_text.append(row['Japanese'])
    en_train_text.append(row['English'])
    row['English'] = "\\" + " " + row['English']
    #ここの感覚は調整する
    #trainとval　を同じデータ、シャッフルするだけにしたらどうなるのか
    train_pairs.append((row['Japanese'],row['English']))
    val_pairs.append((row['Japanese'],row['English']))

random.shuffle(val_pairs)

In [4]:
#適当に確認

#print(ja_train_text[700])
#print(en_train_text[700])
#print(train_pairs)
#print(val_pairs)

In [ ]:
#データのシャッフルはいらない
#とりあえずcsvにある約5000のデータで学習させる


In [5]:
#あとで数値をいじる
vocab_size = 10000
sequence_length = 20

vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)

vectorization.adapt(ja_train_text)
vectorization.adapt(en_train_text)
#print(ja_train_text)
#print(en_train_text)

In [6]:
#データセットの件数が数百件程度であれば32, 64、数万件程度であれば1024, 2048　でtry


batch_size = 128

def format_dataset(ja, en):
    ja = vectorization(ja)
    en = vectorization(en)
    #ここでスライスするかどうかで変えられる
    '''
    japanese"、"english"にしないとエラー
    '''
    return ({"japanese": ja,
        "english": en,
    }, en)


def make_dataset(pairs):
    ja_texts, en_texts = zip(*pairs)
    ja_texts = list(ja_texts)
    en_texts = list(en_texts)
    dataset = tf.data.Dataset.from_tensor_slices((ja_texts, en_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="japanese")

x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

past_target = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [11]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['japanese'].shape: {inputs['japanese'].shape}")
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"targets.shape: {targets.shape}")

#format_datasetのところで調整可能 

inputs['japanese'].shape: (128, 20)
inputs['english'].shape: (128, 20)
targets.shape: (128, 20)


In [12]:
#RNNでseq2seqで訓練

seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])


#損失関数の値が収束するような、epochs数に調整を行う。
seq2seq_rnn.fit(train_ds, 
                epochs=10, 
                validation_data=val_ds)

Epoch 1/10


KeyboardInterrupt: ignored

In [ ]:
'''
翻訳のためのエンコーダーとデコーダの作成をする。
文をいれたら翻訳して出力する。
'''